In [2]:
from libsvm.svmutil import *
from liblinear.liblinearutil import *
import numpy as np
import pandas as pd
import itertools
import random
import sys

In [ ]:
df_tr = pd.read_csv("ML_FP_train.csv")
df_tr.shape #(17170, 29)
random_rows = df_tr.sample(n=10)

In [9]:
df_tr.describe()

,Danceability,Energy,Key,Loudness,Speechiness,Acousticness,Instrumentalness,Liveness,Valence,Tempo,Duration_ms,Views,Likes,Stream,id,Comments
count,17170.000000,1.461500e+04,14568.000000,14601.000000,14655.000000,1.459300e+04,14569.000000,14635.000000,14596.000000,14608.000000,1.456700e+04,1.457400e+04,1.459100e+04,1.460200e+04,17170.000000,1.458900e+04
mean,4.586022,3.358392e-01,5.231604,-7.561370,0.095462,1.148404e-01,0.054934,0.033745,0.525842,120.937819,2.228307e+05,8.791096e+07,6.485064e+05,1.290426e+08,8584.500000,2.810321e+04
std,2.858980,2.426825e-01,3.456986,4.540944,0.103068,2.215160e-01,0.191392,0.114862,0.244912,29.665225,1.211659e+05,2.523918e+08,1.726157e+06,2.417681e+08,4956.696396,2.142922e+05
min,0.000000,8.365427e-15,0.000000,-46.251000,0.000000,1.367631e-18,0.000000,0.000003,0.000000,0.000000,3.098500e+04,2.600000e+01,0.000000e+00,6.574000e+03,0.000000,0.000000e+00
25%,2.000000,1.342177e-01,2.000000,-8.655000,0.036100,9.797218e-05,0.000000,0.000836,0.336000,97.041000,1.793055e+05,1.971684e+06,2.251600e+04,1.628195e+07,4292.250000,5.340000e+02
50%,5.000000,2.954083e-01,5.000000,-6.472000,0.051400,6.967871e-03,0.000002,0.001907,0.531000,119.995500,2.120300e+05,1.427208e+07,1.239210e+05,4.550224e+07,8584.500000,3.171000e+03
75%,7.000000,5.062616e-01,8.000000,-4.920000,0.106000,1.025032e-01,0.000392,0.012487,0.722000,139.990250,2.495115e+05,6.603988e+07,5.077155e+05,1.256493e+08,12876.750000,1.361900e+04
max,9.000000,1.000000e+00,10.000000,0.829000,0.964000,9.880479e-01,1.000000,1.000000,0.993000,243.372000,4.581483e+06,5.773798e+09,4.014762e+07,3.386520e+09,17169.000000,1.608314e+07
